### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_items.shape

(876365, 4)

In [15]:
user_items_mod = user_items.iloc[:576365,:]
user_items_mod.shape

(576365, 4)

In [16]:
user_by_movie = user_items_mod.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 7.  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [19]:
ratings_mat.shape
(~np.isnan(ratings_mat)).sum()

56

In [34]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = (~np.isnan(ratings_mat)).sum() # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(n_movies,latent_features) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
#     preds = np.dot(user_mat, movie_mat)
#     print("The initial random predicted value for the missing rating is {}:".format(preds[0,0]))

    # for each iteration
    for it in range(0,iters):
        
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for u in range(0,n_users):
            for m in range(0,n_movies):
                
                # if the rating exists
                if ~np.isnan(ratings_mat[u,m]) and ratings_mat[u,m]>0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    err = ratings_mat[u,m] - np.dot(user_mat[u,:],movie_mat[:,m])
                    
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += err**2
                    
                    # update the values in each matrix in the direction of the gradient
                    pr = 2*learning_rate*err
                    for l in range(0,latent_features):
                        user_mat[u,l] = user_mat[u,l]+pr*movie_mat[l,m]
                        movie_mat[l,m] = movie_mat[l,m]+pr*user_mat[u,l]

        # print results
        print("%d \t\t %f" % (it+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [35]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10) 
# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 48.541293
2 		 22.917304
3 		 7.206152
4 		 3.303856
5 		 2.764164
6 		 2.652799
7 		 2.589691
8 		 2.532067
9 		 2.471295
10 		 2.404199


In [36]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.04378287  7.37813153  9.26000231  8.38333636]
 [ 9.71510243  8.6396911  10.46286604  9.39081831]
 [ 6.88773306  6.09409145  6.42929873  6.6952351 ]
 [ 7.61360616  6.33206852  6.96792551  7.17421438]
 [ 9.74952093  8.51790047 10.15297584  9.35525943]
 [ 9.07318453  7.36953412  9.02008202  8.40692156]
 [ 9.20979879  8.30754022  9.63002873  8.98658183]
 [ 7.00324609  5.73143263  7.45953752  6.48609763]
 [ 9.34764891  8.24722043  9.83154869  9.01083378]
 [10.00396398  8.50572823  9.47260546  9.51646904]
 [ 8.21882313  6.88315645  9.2035449   7.67763025]
 [10.17258926  8.48663224 10.07190313  9.55787616]
 [ 7.9930037   6.6829907   7.96076423  7.51581001]
 [ 8.79205624  7.98183339  9.53312539  8.58949179]]
[[nan  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]]


**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [37]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 47.233011
2 		 21.364263
3 		 6.597449
4 		 3.269939
5 		 2.803926
6 		 2.692610
7 		 2.635345
8 		 2.588751
9 		 2.541398
10 		 2.489295
11 		 2.430662
12 		 2.364445
13 		 2.289921
14 		 2.206640
15 		 2.114475
16 		 2.013703
17 		 1.905080
18 		 1.789890
19 		 1.669933
20 		 1.547446
21 		 1.424951
22 		 1.305034
23 		 1.190103
24 		 1.082156
25 		 0.982617
26 		 0.892258
27 		 0.811226
28 		 0.739152
29 		 0.675302
30 		 0.618745
31 		 0.568494
32 		 0.523616
33 		 0.483293
34 		 0.446848
35 		 0.413747
36 		 0.383579
37 		 0.356033
38 		 0.330871
39 		 0.307905
40 		 0.286976
41 		 0.267945
42 		 0.250676
43 		 0.235040
44 		 0.220904
45 		 0.208138
46 		 0.196612
47 		 0.186199
48 		 0.176779
49 		 0.168234
50 		 0.160460
51 		 0.153358
52 		 0.146839
53 		 0.140825
54 		 0.135247
55 		 0.130043
56 		 0.125163
57 		 0.120562
58 		 0.116201
59 		 0.112050
60 		 0.108081
61 		 0.104273
62 		 0.100608
63 		 0.097070
64 		

In [38]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 8.76385032  7.00036084 10.00010903  7.99963387]
 [ 9.99668384  7.99820842  9.99966148 10.00346597]
 [ 5.00100803  8.00056995  5.000129    7.99896401]
 [10.00174543  8.00096822  1.00019809  9.99818395]
 [ 8.99900019  9.99945031  9.99989938  9.00107008]
 [ 9.99949403  3.99973049  8.99996749 10.00056406]
 [ 8.99408675  7.99684105  9.9994394   9.00620219]
 [ 5.99797473  3.99895717  9.99987927  6.00220787]
 [10.00267089  9.00147079 10.00035241  7.99734183]
 [ 9.9902894   9.99483269  8.99909565  9.01018086]
 [ 8.00331412  7.0018662  10.00048319  6.99672223]
 [ 9.00549148  9.00301964 10.00067964  9.99443362]
 [ 9.00068193  5.00036303  9.00011137  6.99938636]
 [ 9.0049687   8.00264507 10.00052652  7.99492596]]
[[nan  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]]


**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [39]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan,  7., 10.,  8.],
        [10.,  8., 10., 10.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8.,  1., 10.],
        [ 9., 10., 10.,  9.],
        [10.,  4.,  9., 10.],
        [ 9.,  8., 10.,  9.],
        [ 6.,  4., 10.,  6.],
        [10.,  9., 10.,  8.],
        [10., 10.,  9.,  9.],
        [ 8.,  7., 10.,  7.],
        [ 9.,  9., 10., 10.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.]])

In [40]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)
#use your function wit #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 47.168652
2 		 22.153049
3 		 7.176976
4 		 3.427893
5 		 2.876575
6 		 2.772030
7 		 2.730137
8 		 2.700163
9 		 2.671535
10 		 2.640943
11 		 2.606916
12 		 2.568500
13 		 2.524881
14 		 2.475290
15 		 2.418978
16 		 2.355249
17 		 2.283501
18 		 2.203294
19 		 2.114424
20 		 2.017010
21 		 1.911573
22 		 1.799096
23 		 1.681043
24 		 1.559327
25 		 1.436213
26 		 1.314149
27 		 1.195562
28 		 1.082637
29 		 0.977127
30 		 0.880227
31 		 0.792542
32 		 0.714127
33 		 0.644598
34 		 0.583272
35 		 0.529304
36 		 0.481801
37 		 0.439910
38 		 0.402858
39 		 0.369979
40 		 0.340708
41 		 0.314575
42 		 0.291184
43 		 0.270205
44 		 0.251353
45 		 0.234380
46 		 0.219067
47 		 0.205220
48 		 0.192662
49 		 0.181236
50 		 0.170801
51 		 0.161230
52 		 0.152411
53 		 0.144245
54 		 0.136646
55 		 0.129541
56 		 0.122866
57 		 0.116567
58 		 0.110599
59 		 0.104926
60 		 0.099516
61 		 0.094345
62 		 0.089393
63 		 0.084645
64 		

In [41]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 7.58449959099862:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [42]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

SyntaxError: invalid syntax (<ipython-input-42-4d8e15aff68b>, line 5)

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [ ]:
# Replace each of the comments below with the correct values
num_ratings = # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.isnan(first_1000_users).sum() # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")